In [2]:
#!pip install pyspark
import pyspark

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("group_agg").getOrCreate()


In [43]:
import random
staff = ["Alice", "Bob", "Charlie", "David", "Eva", "Frank", "Grace", "Henry", "Isabel", "Jack"]
age = list(random.randint(30, 65) for _ in range(10))
salary = list(random.randint(40000, 90000) for _ in range(10))
department = ["Math", "Physics", "Human Resource", "Physics", "Math", "Physics", "Math", "Human Resource", "Math", "Math"]

data = list(zip(staff, age, salary, department))

data = spark.createDataFrame(data, ["Name", "Age", "Salary", "Department"])
data.show()

+-------+---+------+--------------+
|   Name|Age|Salary|    Department|
+-------+---+------+--------------+
|  Alice| 41| 47522|          Math|
|    Bob| 37| 63682|       Physics|
|Charlie| 40| 89893|Human Resource|
|  David| 42| 45607|       Physics|
|    Eva| 39| 60491|          Math|
|  Frank| 30| 80255|       Physics|
|  Grace| 33| 42959|          Math|
|  Henry| 40| 77280|Human Resource|
| Isabel| 30| 84224|          Math|
|   Jack| 38| 67555|          Math|
+-------+---+------+--------------+



In [44]:
data.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: long (nullable = true)
 |-- Salary: long (nullable = true)
 |-- Department: string (nullable = true)



In [55]:
data.select(["Department", "Salary"]).groupBy("Department").mean().show()

+--------------+------------------+
|    Department|       avg(Salary)|
+--------------+------------------+
|          Math|           60550.2|
|Human Resource|           83586.5|
|       Physics|63181.333333333336|
+--------------+------------------+



In [56]:
data.groupBy("Department").count().show()

+--------------+-----+
|    Department|count|
+--------------+-----+
|          Math|    5|
|Human Resource|    2|
|       Physics|    3|
+--------------+-----+



In [66]:
mean_salary = data.select(["Department", "Salary"]).groupBy("Department").mean()
mean_salary.orderBy("avg(Salary)").show()

+--------------+------------------+
|    Department|       avg(Salary)|
+--------------+------------------+
|          Math|           60550.2|
|       Physics|63181.333333333336|
|Human Resource|           83586.5|
+--------------+------------------+



In [71]:
data.orderBy("Salary").show()

+-------+---+------+--------------+
|   Name|Age|Salary|    Department|
+-------+---+------+--------------+
|  Grace| 33| 42959|          Math|
|  David| 42| 45607|       Physics|
|  Alice| 41| 47522|          Math|
|    Eva| 39| 60491|          Math|
|    Bob| 37| 63682|       Physics|
|   Jack| 38| 67555|          Math|
|  Henry| 40| 77280|Human Resource|
|  Frank| 30| 80255|       Physics|
| Isabel| 30| 84224|          Math|
|Charlie| 40| 89893|Human Resource|
+-------+---+------+--------------+



In [81]:
# problem Who is aged under 40 and is the youngest, math, earning the highest

math_youngsters = data.filter((data["Department"] == "Math") & (data["Age"] <= 40))
math_youngsters.orderBy(math_youngsters["Salary"]).show()

+------+---+------+----------+
|  Name|Age|Salary|Department|
+------+---+------+----------+
| Grace| 33| 42959|      Math|
|   Eva| 39| 60491|      Math|
|  Jack| 38| 67555|      Math|
|Isabel| 30| 84224|      Math|
+------+---+------+----------+



In [87]:
data.select(["Age", "Department", "Salary"]).groupBy("Department").avg().show()

+--------------+------------------+------------------+
|    Department|          avg(Age)|       avg(Salary)|
+--------------+------------------+------------------+
|          Math|              36.2|           60550.2|
|Human Resource|              40.0|           83586.5|
|       Physics|36.333333333333336|63181.333333333336|
+--------------+------------------+------------------+



In [100]:
data.show()

data.orderBy(["Department", "Salary", "Age"]).show()

+-------+---+------+--------------+
|   Name|Age|Salary|    Department|
+-------+---+------+--------------+
|  Alice| 41| 47522|          Math|
|    Bob| 37| 63682|       Physics|
|Charlie| 40| 89893|Human Resource|
|  David| 42| 45607|       Physics|
|    Eva| 39| 60491|          Math|
|  Frank| 30| 80255|       Physics|
|  Grace| 33| 42959|          Math|
|  Henry| 40| 77280|Human Resource|
| Isabel| 30| 84224|          Math|
|   Jack| 38| 67555|          Math|
+-------+---+------+--------------+

+-------+---+------+--------------+
|   Name|Age|Salary|    Department|
+-------+---+------+--------------+
|  Henry| 40| 77280|Human Resource|
|Charlie| 40| 89893|Human Resource|
|  Grace| 33| 42959|          Math|
|  Alice| 41| 47522|          Math|
|    Eva| 39| 60491|          Math|
|   Jack| 38| 67555|          Math|
| Isabel| 30| 84224|          Math|
|  David| 42| 45607|       Physics|
|    Bob| 37| 63682|       Physics|
|  Frank| 30| 80255|       Physics|
+-------+---+------+-------